In [2]:
import os 

import numpy as np 
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt

from tensorflow import keras

# MLflow dashboard
import mlflow
mlflow.set_tracking_uri('http://35.228.45.76:5000')
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='../../keys/mlflow-312506-8cfad529f4fd.json'

# Import data augmentation
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from augmentation.methods import *

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [3]:
import warnings
warnings.simplefilter('ignore')

In [4]:
random_state = 47
np.random.seed(random_state)

In [5]:
physical_devices = tf.config.experimental.list_physical_devices( 'GPU' )
print( 'Num GPUs Available: ', len( physical_devices ) )
if len( physical_devices ) > 0:
    tf.config.experimental.set_memory_growth( physical_devices[0], True )

Num GPUs Available:  0


# 1. Data Preparation
## 1.1 Load the Posenet files 


In [9]:
data_path = '../../datasets/'
files = [
    'posenet_scored_all_1.csv',
    'posenet_scored_all_2.csv',
    'posenet_scored_all_3.csv',
    'posenet_scored_all_4.csv',
]
df = None

In [10]:
for file in files:
    try:
        dataset = pd.read_csv(data_path + file)

        if df is None:
            df = dataset
        else:
            df = df.append(dataset, ignore_index=True)  

    except IOError as e:
        print('Error in reading file: ', e)

print(df.shape)

(225092, 54)


In [11]:
X = df.drop(columns=['ConfidenceScore', 'GoodnessScore'])
y = df[['ConfidenceScore', 'GoodnessScore']]

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=random_state)

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

print('Training labels shape:', y_train.shape)
print('Test labels shape:', y_test.shape, '\n')

print('Training features shape:', X_train.shape)
print('Test features shape:', X_test.shape)

Training labels shape: (202582, 2)
Test labels shape: (22510, 2) 

Training features shape: (202582, 52)
Test features shape: (22510, 52)
